# This code is for ModelA
build model  for each target country without transfer

In [10]:
# Part 1 - Data Preprocessing
import tensorflow as tf
# Importing the libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import os
from tqdm import tqdm
import datetime 

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.models import Model

from keras.layers import Input, Embedding, LSTM, Dense,  Lambda

from keras.backend import slice
from keras.constraints import max_norm
import warnings
warnings.simplefilter('ignore')
os.environ["CUDA_VISIBLE_DEVICES"]='-1'

##　locate the directory storing the data 
os.chdir(os.getcwd()+'/data/')

In [11]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [12]:
def create_sequences_x(data, seq_length):
    xs = []
    for i in range(len(data)-seq_length+1):
        x = data[i:(i+seq_length)]
        xs.append(x)
    return np.array(xs)

def create_sequences_y(data, seq_length):
    ys = []
    for i in range(seq_length, len(data)):
        y = data[i]
        ys.append(y)
    return np.array(ys)

def MAPE(y, y_pred):
    mape = sum(abs(y-y_pred)/y)/len(y)
    print('MAPE: ', mape)
    return mape

In [13]:
from os import listdir
from os.path import isfile, join
from re import sub
mypath = 'target/'
# target_countries = ['Albania','Algeria','Argentina','Armenia','Australia','Azerbaijan','Bangladesh','Belarus','Belgium','Bermuda',
#              'Bolivia','Brazil','Bulgaria','Canada','Chile','Colombia','Costa Rica','Cuba','Czech Republic','Denmark',
#              'El Salvador','Estonia','Finland','France','Ghana','Gibraltar','Greece','Honduras','Hungary','India',
#              'Indonesia','Iran','Iraq','Ireland','Israel','Jamaica','Jordan','Liberia','Luxembourg','Malaysia','Mexico',
#              'Morocco','Nepal','Netherlands','Nigeria','Pakistan','Paraguay','Peru','Philippines','Poland','Portugal',
#              'Qatar','Republic of the Congo','Romania','Russia','Rwanda','Saudi Arabia','Senegal','Sierra Leone',
#              'Singapore','Slovakia','Slovenia','South Africa','Sri Lanka','Switzerland','Thailand','Tunisia','Turkey',
#              'Ukraine','United Kingdom','United States','Venezuela']

target_countries = ['Australia','France','Greece','Iraq','Netherlands']

In [14]:
ret_test = pd.DataFrame(index=target_countries, columns=['MAPE'])
no_transfer = pd.DataFrame(index=['loop1','loop2','loop3','loop4','loop5'], columns=['MAPE(mean)','MAPE(std)','MAPE<0.1','MAPE<0.05'])

In [16]:
pred_length = 7
seq_length = 7
dict_res = {}
for l in range(1):
    for tar in target_countries:
        print(tar)
        # data preprocessing
        df = pd.read_excel('target/'+tar+'.xlsx', index_col=0)

        df_new_1day = df.diff(periods=1)
        df_new_1day.rename(columns={'confirmed cases per million':'new cases'}, inplace=True)

        df_new_7days = df[['confirmed cases per million']].diff(periods=pred_length)
        df_new_7days.rename(columns={'confirmed cases per million':'new cases'}, inplace=True)
        df_new_7days['cum cases'] = 0
        df_new_7days['cum cases'][pred_length:] = df['confirmed cases per million'].values[0:len(df_new_7days)-pred_length]

        x_seq0 = create_sequences_x(np.array(df_new_1day.dropna()), seq_length)
        y_seq0 = create_sequences_y(np.array(df_new_7days.dropna()), seq_length)
        x_seq0 = x_seq0[0:len(y_seq0)]
        x_seq1 = np.reshape(x_seq0, newshape=(-1,1))
        scaler_x = MinMaxScaler()
        scaler_y = MinMaxScaler()

        scaler_x = scaler_x.fit(x_seq1)
        x = scaler_x.transform(x_seq1)
        x = np.reshape(x, newshape=(x_seq0.shape))
        y_seq1 = np.reshape(y_seq0[:,0:1], newshape=(-1,1))
        scaler_y = scaler_y.fit(y_seq1)
        y = scaler_y.transform(y_seq1)
        y = np.reshape(y, newshape=(y_seq0[:,0:1].shape))

        test_idx = int(len(x)*0.8)

        X_train = x[0:test_idx, :, 0:1].copy()
        y_train = np.reshape(y[0:test_idx], (-1))
        X_test = x[test_idx:,:,0:1].copy()
        y_test = np.reshape(y[test_idx:], (-1))

        # Building the RNN
        main_input = Input(shape=(seq_length,1,), dtype='float32', name='main_input')  
        lstm_out = LSTM(16,activation='relu')(main_input)    
        main_output = Dense(units = 1,activation='sigmoid')(lstm_out)
        regressor = Model(inputs=main_input, outputs=main_output)

        regressor.compile(optimizer='adam', loss='mse')
        # Fitting the RNN to the Training set
        regressor.fit(X_train, y_train, epochs = 7, batch_size = 1, verbose=0)

        # Predicting daily cases
        predicted_cases = regressor.predict(X_test)
        predicted_cases = scaler_y.inverse_transform(predicted_cases)

        true_cases = np.reshape(scaler_y.inverse_transform(np.reshape(y_test,(-1,1))),(-1))+y_seq0[test_idx:,1]
        predicted_cases = np.reshape(predicted_cases,(-1))+y_seq0[test_idx:,1]
        mape = MAPE(true_cases, predicted_cases)

        ret_test.loc[tar, 'MAPE'] = mape
    no_transfer.loc['loop'+str(l+1), 'MAPE(mean)'] = ret_test.MAPE.mean() 
    no_transfer.loc['loop'+str(l+1), 'MAPE(std)'] = ret_test.MAPE.std()
    no_transfer.loc['loop'+str(l+1), 'MAPE<0.1'] = ret_test[(ret_test.MAPE<0.1)].shape[0]
    no_transfer.loc['loop'+str(l+1), 'MAPE<0.05']=ret_test[(ret_test.MAPE<0.05)].shape[0]

Australia
MAPE:  0.10787832782782499
France
MAPE:  0.08646141231728575
Greece
MAPE:  0.09588690084319153
Iraq
MAPE:  0.16491545188966034
Netherlands
MAPE:  0.09075611504388666


In [9]:
no_transfer.to_csv('../result/modelA_notransfer.csv')